# 00 - Environment Setup

This is the notebook that sets up the GCP project  

---
## Setup

inputs:

In [ ]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

In [ ]:
REGION = 'us-central1'

packages:

In [ ]:
from google.cloud import storage
from google.cloud import bigquery

import pandas as pd
from sklearn import datasets

clients:

In [ ]:
gcs = storage.Client(project = PROJECT_ID)
bq = bigquery.Client(project = PROJECT_ID)

parameters:

In [ ]:
BUCKET = PROJECT_ID

---
## Create Storage Bucket
Check to see if bucket already exist and create if missing:
- [GCS Python Client](https://cloud.google.com/python/docs/reference/storage/latest/google.cloud.storage.client.Client)

In [ ]:
if not gcs.lookup_bucket(BUCKET):
    bucketDef = gcs.bucket(BUCKET)
    bucket = gcs.create_bucket(bucketDef, project=PROJECT_ID, location=REGION)
    print(f'Created Bucket: {gcs.lookup_bucket(BUCKET).name}')
else:
    bucketDef = gcs.bucket(BUCKET)
    print(f'Bucket already exist: {bucketDef.name}')

In [ ]:
print(f'Review the storage bucket in the console here:\nhttps://console.cloud.google.com/storage/browser/{PROJECT_ID};tab=objects&project={PROJECT_ID}')

---
<a id = 'permissions'></a>
## Service Account & Permissions

This notebook instance is running as a service account in GCP.  This service account will also be used to run other services in Vertex AI like training jobs and pipelines.  The service account will need permission to interact with object in Cloud Storage which requires the role ([roles/storage.objectAdmin](https://cloud.google.com/storage/docs/access-control/iam-roles)).  

Get the current service account:

In [ ]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

Enable the Cloud Resource Manager API:

In [ ]:
!gcloud services enable cloudresourcemanager.googleapis.com

List the service accounts current roles:

In [ ]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"

If the resulting list is missing `roles/storage.objectAdmin` or another role that contains this permission, like the basic role `roles/owner`, then it will need to be added for the service account. Use these instructions to complete this:

In [ ]:
print(f'Go To IAM in the Google Cloud Console:\nhttps://console.cloud.google.com/iam-admin/iam?orgonly=true&project={PROJECT_ID}&supportedpurview=organizationId')

From the console link above, or by going to https:/console.cloud.google.com and navigating to "IAM & Admin > IAM":
- Locate the row for the service account listed above: `<project number>-compute@developer.gserviceaccount.com`
- Under the `inheritance` column click the pencil icon to edit roles
- In the fly over menu, under `Assign roles` select `Add Another Role`
- Click the `Select a role` box and type `Storage Object Admin`, then select `Storage Object Admin`
- Click Save
- Rerun the list of services below and verify the role has been added:

In [ ]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"

---
## Install KFP
If you get an error after a step, rerun it.  The dependecies sometimes resolve.
- [Install the Kubeflow Pipelines SDK](https://www.kubeflow.org/docs/components/pipelines/v1/sdk/install-sdk/)

In [ ]:
!pip install kfp -U -q

In [ ]:
!pip install google-cloud-pipeline-components -U -q

## Update AIPlatform Package:

The `google-cloud-aiplatform` package updates frequently.  Update it for latest functionality.

- [aiplatform Python Client](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform)
- [GitHub Repo for api-common-protos](https://github.com/googleapis/api-common-protos)


In [ ]:
!pip install googleapis-common-protos -U -q

In [ ]:
!pip install google-cloud-aiplatform -U -q

In [ ]:
!pip install google-cloud-aiplatform

In [ ]:
#from google.cloud import aiplatform
#aiplatform.__version__